### Homework 1: Introduction

In this homework, we'll learn more about sarch and use Elastic Search for practice.

You will need to install the following packages if you do not already have them:
`pip install requests elasticsearch tqdm tiktoken`

In [1]:
import os
import sys
import requests

import openai
from openai import OpenAI
from elasticsearch import Elasticsearch
import tqdm
from tqdm.auto import tqdm
import tiktoken

### Q1. Running Elastic

Run Elastic Search 8.17.6, and get the cluster information.  If you run it on localhost, this is how you do it:

```
curl localhost:9200
```

##### What's the ```version.build_hash``` value?

### A1.  build_hash:	"dbcbbbd0bc4924cfeb28929dc05d82d662c527b7"

### Getting the Data

Now let's get the FAQ data.  You can run this snippet:

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
len(documents)

948

### Q2. Indexing the Data

Index the data in the same way as was shown in the course videos.  Make the `course` field a keyword and the rest should be text.

Which function do you use for adding your data to elastic?

* insert
* index
* put
* add

### A2. To add your data to elastic you need to use the `index` function.

### Start elasticsearch in docker

```
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.17.6
```

In [8]:
es_client = Elasticsearch('http://localhost:9200') 

In [28]:
es_client.info()

ObjectApiResponse({'name': '813b77b768e6', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'phE7t7bbQ6i8bF8NBlScmA', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [10]:
index_name = "course-questions"

Note: If you need to delete an index, the syntax is `es_client.indices.delete(index=index_name)`

In [11]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Q3. Searching

Now let's search in our index.

We will execute a query "How do you execute a command on a Kubernetes pod?".

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 84.50
* 64.50
* 44.50
* 24.50

Look at the `_score` field.

Note: For boosting info, see https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-boosting-query.html

In [61]:
def search(query, index_name="course-questions", max_results=5):

    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es.search(index=index_name, body=search_query)
    for hit in response['hits']['hits']:
        doc = hit['_source']
        print(f"Question: {doc['question']}\nAnswer: {doc['text']}\n\n")
        print(f"Score: {hit['_score']}\n")


In [62]:
search(query="How do execute a command on a Kubernetes pod?", index_name="course-questions", max_results=1)


Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Score: 44.50556



### A3. The score is 44.5.

### Q4. Filtering

Now ask a different question: "How do you copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results.  What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container's working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [63]:
def retrieve_filtered_docs(query, index_name="course-questions", max_results=5):
    
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents


In [65]:
documents = retrieve_filtered_docs(query="How do you copy a file to a Docker container?", index_name="course-questions", max_results=3)
documents


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

### A4. The third response is "How do I copy files from a different folder into docker container's working directory?".

### Q5. Building a Prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context.  Separate context entries by two linebreaks. (```\n\n```)

```
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

In [66]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [ ]:
def apply_context_template(doc):
    question = doc.get('question', 'No question')
    text = doc.get('text', 'No text')
    return context_template.format(question=question, text=text)

In [68]:
def build_context(docs):
    return "\n\n".join(apply_context_template(doc) for doc in docs)

In [69]:
context = build_context(documents)

In [70]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [71]:
user_question = "How do I execute a command in a running docker container?"

In [72]:
prompt = prompt_template.format(question=user_question, context=context)
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dire

In [73]:
len(prompt)

1462

What's the length of the resulting prompt?  (Use the `len` function)

* 946
* 1446
* 1946
* 2446

### A5. The closest prompt length is 1446.

### Q6. Tokens

WHen we use the OpenAI Platform, we're charged by the umber of tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

Let's calculate the number of tokens in our query:

`encoding = tiktoken.encoding_for_model("gpt-4o")`

Use the `encode` function.  How many tokens does our prompt have?

* 120
* 220
* 320
* 420

Note: To decode back a token into a work, you can use the `decode_single_token_bytes` function:

`encoding.decode_single_token_bytes(63842)`

In [74]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [75]:
encoding.encode(prompt)

[63842,
 261,
 4165,
 14029,
 29186,
 13,
 30985,
 290,
 150339,
 4122,
 402,
 290,
 31810,
 8099,
 591,
 290,
 40251,
 7862,
 13,
 7649,
 1606,
 290,
 19719,
 591,
 290,
 31810,
 8099,
 1261,
 55959,
 290,
 150339,
 364,
 107036,
 25,
 3253,
 621,
 357,
 15792,
 261,
 6348,
 306,
 261,
 6788,
 62275,
 9282,
 1715,
 10637,
 50738,
 734,
 48,
 25,
 3253,
 621,
 357,
 15199,
 261,
 62275,
 9282,
 3901,
 32,
 25,
 41281,
 290,
 9282,
 3621,
 306,
 25383,
 6766,
 326,
 151187,
 290,
 7251,
 4859,
 11,
 813,
 484,
 480,
 13217,
 261,
 38615,
 6348,
 558,
 68923,
 2461,
 533,
 278,
 2230,
 7962,
 4859,
 38615,
 464,
 3365,
 523,
 3335,
 290,
 9282,
 382,
 4279,
 6788,
 11,
 15792,
 261,
 6348,
 306,
 290,
 4857,
 9282,
 734,
 68923,
 10942,
 350,
 6555,
 290,
 9282,
 26240,
 446,
 68923,
 25398,
 533,
 278,
 464,
 6896,
 26240,
 29,
 38615,
 198,
 6103,
 277,
 10732,
 391,
 79771,
 1029,
 48,
 25,
 3253,
 621,
 357,
 5150,
 6291,
 591,
 922,
 2698,
 7342,
 316,
 62275,
 9282,
 3901,
 32,
 25

In [76]:
len(encoding.encode(prompt))

322

In [77]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [78]:
encoding.decode_single_token_bytes(62275)

b' docker'

### A6. The closest length of the encoded propmpt is 320.

### Bonus: Generating the Answer (ungraded)

Let's send the prompt to OpenAI.  What's the response?

Note: You can replace OpenAI with Ollama.  See module 2.

In [79]:
import os
from openai import OpenAI

In [80]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)

In [81]:
messages = [
        {"role": "system", "content": "You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION."},
        {"role": "user", "content": prompt}
    ]


In [82]:
openai_response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages #,
    #max_tokens=150
)

In [83]:
reply = openai_response.choices[0].message.content.strip()
reply

'To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID by listing the running containers with `docker ps`. Once you have the container ID, you can execute a command, such as starting a bash shell, using the following syntax:\n\n```bash\ndocker exec -it <container-id> bash\n```\n\nThis allows you to run commands inside the running container.'

In [84]:
print("Reply:", reply)

Reply: To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID by listing the running containers with `docker ps`. Once you have the container ID, you can execute a command, such as starting a bash shell, using the following syntax:

```bash
docker exec -it <container-id> bash
```

This allows you to run commands inside the running container.


### Bonus: Calculating the Costs (ungraded)

Suppose that on average per request we send 150 token and receive back 250 tokens.

How much will it cost to run 1,000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17th, the prices for gpt4o are:
* Input: $0.005 / 1K tokens
* output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [85]:
openai_response.usage

CompletionUsage(completion_tokens=87, prompt_tokens=365, total_tokens=452, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [86]:
# number of input tokens
num_input_tokens = (1000 * 365) 

In [87]:
# number of total tokens
num_output_tokens = (1000 * 452)


In [88]:
cost = ((0.005 * num_input_tokens)/1000) + ((0.015 * num_output_tokens)/1000)
print(f'Total cost of running 1,000 requests ${cost}')

Total cost of running 1,000 requests $8.605
